# Restricted Hartree-Fock

## Theory

In this tutorial, we will seek to introduce the theory and implementation of the quantum chemical method known as Hartree-Fock Self-Consistent Field Theory (HF-SCF) with restricted orbitals and closed-shell systems (RHF).  This theory seeks to solve the pseudo-eigenvalue matrix equation 

$$\sum_{\nu} F_{\mu\nu}C_{\nu i} = \epsilon_i\sum_{\nu}S_{\mu\nu}C_{\nu i}$$
$${\bf FC} = {\bf SC\epsilon},$$

called the Roothan equations, which can be solved self-consistently for the orbital coefficient matrix **C** to and the orbital energy eigenvalues $\epsilon_i$.  The Fock matrix, **F**, has elements $F_{\mu\nu}$ given (in the atomic orbital basis) as

$$F_{\mu\nu} = H_{\mu\nu} + 2(\mu\,\nu\left|\,\lambda\,\sigma)D_{\lambda\sigma} - (\mu\,\lambda\,\right|\nu\,\sigma)D_{\lambda\sigma},$$

where $D_{\lambda\sigma}$ is an element of the one-particle density matrix **D**, constructed from the orbital coefficient matrix **C**:

$$D_{\lambda\sigma} = C_{\sigma i}C_{\lambda i}$$

Formally, the orbital coefficient matrix **C** is a $N\times M$ matrix, where $N$ is the number of atomic basis functions, and $M$ is the total number of molecular orbitals.  Physically, this matrix describes the contribution of every atomic basis function (columns) to a particular molecular orbital (e.g., the $i^{\rm th}$ row).  The density matrix **D** is a square matrix describing the electron density contained in each orbital.  In the molecular orbital basis, the density matrix has elements

$$D_{pq} = \left\{\begin{array}{ll} 2\delta_{pq} & p\; {\rm occupied} \\ 0 & p\; {\rm virtual} \\ \end{array}\right .$$

The total RHF energy is given by

$$E^{\rm RHF}_{\rm total} = E^{\rm RHF}_{\rm elec} + E^{\rm BO}_{\rm nuc},$$

where $E^{\rm RHF}_{\rm elec}$ is the final electronic RHF energy, and $E^{\rm BO}_{\rm nuc}$ is the total nuclear repulsion energy within the Born-Oppenheimer approximation.  To compute the electronic energy, we may use the density matrix in the AO basis:

$$E^{\rm RHF}_{\rm elec} = (F_{\mu\nu} + H_{\mu\nu})D_{\mu\nu},$$

and the nuclear repulsion energy is simply

$$E^{\rm BO}_{\rm nuc} = \sum_{A>B}\frac{Z_AZ_B}{r_{AB}}$$

where $Z_A$ is the nuclear charge of atom $A$, and the sum runs over all unique nuclear pairs.

## Implementation

Using the above overview, let's write a RHF program using PySCF and NumPy.  First, we need to import these Python modules: 

In [1]:
# Check if notebook is running on Colab
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
    
if IN_COLAB:
    !pip install pyscf # If asked, restart runtime after PySCF has been installed

In [2]:
# ==> Import Psi4 & NumPy <==
from pyscf import gto, scf
import numpy as np
import scipy

Next, we construct a water molecule using a cc-pVDZ [basis set](https://en.wikipedia.org/wiki/Basis_set_(chemistry)).


In [3]:
mol = gto.M(atom = 'O 0.0 0.0 0.0; H 1.0 0.0 0.0; H 0.0 1.0 0.0', basis = 'ccpvdz')

Since we will be writing our own, iterative RHF procedure, we will need to define options that we can use to tweak our convergence behavior.  For example, if something goes wrong and our SCF doesn't converge, we don't want to spiral into an infinite loop.  Instead, we can specify the maximum number of iterations allowed, and store this value in a variable called `maxiter`.  Here are some good default options for our program:
~~~python
MAXITER = 40
E_conv = 1.0e-6
~~~
These are by no means the only possible values for these options, and it's encouraged to try different values and see for yourself how different choices affect the performance of our program.  For now, let's use the above as our default.

In [4]:
MAXITER = 40
E_conv = 1.0e-6

Before we can build our Fock matrix, we'll need to compute the following static one- and two-electron quantities:

- Electron repulsion integrals (ERIs) **ERI** between our AOs in [chemical notation](http://vergil.chemistry.gatech.edu/notes/permsymm/permsymm.html)
- Overlap matrix **S**
- Core Hamiltonian matrix **H**

In [5]:
S = mol.intor('int1e_ovlp')
T = mol.intor('int1e_kin')
V = mol.intor('int1e_nuc')
H_core = T + V
eri = mol.intor('int2e')

We also need the Born-Oppenheimer nuclear repulsion energy, $E^{\rm BO}_{\rm nuc}$ and the number of doubly occupied orbitals (which is equal to the number of $\alpha$-spin electrons).

In [14]:
enuc = mol.get_enuc()
ndocc = mol.nelec[0]

The Roothaan equations

$${\bf FC} = {\bf SC\epsilon}$$

are only *pseudo*-eigenvalue equations due to the presence of the overlap matrix **S** on the right hand side of the equation.  As such, we need to use the generalized eigensolver of [scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigh.html). Before we can get down to the business of using the Fock matrix **F** to compute the RHF energy, we first need to compute the orbital coefficient **C** matrix.  But, before we compute the **C** matrix, we first need to build **F**.  Wait...hold on a second.  Which comes first, **C** or **F**?  Looking at the Roothan equations more closely, we see that that both sides depend on the **C** matrix, since **F** is a function of the orbitals:

$${\bf F}({\bf C}){\bf C} = {\bf SC\epsilon}\,;\;\;F_{\mu\nu} = H_{\mu\nu} + 2(\mu\,\nu\mid\lambda\,\sigma)C_{\sigma i}C_{\lambda i} - (\mu\,\lambda\,\mid\nu\,\sigma)C_{\sigma i}C_{\lambda i}.$$

Therefore technically, *neither* **F** nor **C** can come first!  In order to proceed, we instead begin with a *guess* for the Fock matrix, from which we obtain a guess at the **C** matrix.  Without orbital coefficients (and therefore without electron densities), the most logical starting point for obtaining a guess at the Fock matrix is to begin with the only component of **F** that does *not* involve densities: the core Hamiltonian, $\mathbf{H}_{\text{core}}$.

In [15]:
eps, C = scipy.linalg.eigh(H_core, S)

To construct density matrix $\mathbf{D}$ from the **occupied** molecular orbitals, we'll have to perform a number of so-called **tensor contractions**. These can be implemented using [numpy's einsum](https://numpy.org/doc/stable/reference/generated/numpy.einsum.html). For example, the following matrix multiplication
$$C_{ij} = \sum_{k} A_{ik} * B_{kj}$$
can be implemented as
```python
C = np.einsum('ik,kj->ij', A, B, optimize=True)
```
where we tell `einsum` to optimize the way in which it computes this contraction.

In [16]:
C_occ = C[:, :ndocc]
D = np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)

From this density matrix, we can compute the Coulomb and Exchange matrices **J** and **K**, with elements
\begin{align}
J[D_{rs}]_{pq} &= (p,q \mid r,s)D_{rs}\\
K[D_{rs}]_{pq} &= (p,r \mid q,s)D_{rs},
\end{align}
can be built with
```python
J = np.einsum('pqrs,rs->pq', eri, D, optimize=True)
K = np.einsum('prqs,rs->pq', eri, D, optimize=True)
```
Fortunately, once **J** and **K** have been built, the Fock matrix may be computed as a simple matrix addition

$$ {\bf F} = {\bf H} + 2{\bf J} - {\bf K}.$$

Let's now write our SCF iterations according to the following algorithm:

for scf_iter less than MAXITER, do:
1. Build Fock matrix
    - Build the Coulomb matrix **J** 
    - Build the Exchange matrix **K** 
    - Form the Fock matrix
2. RHF Energy
    - Compute total RHF energy   
    - If change in RHF energy less than E_conv, break    
    - Save latest RHF energy as E_old
3. Compute new orbital guess
    - Diagonalize ${\bf F}$ for $\epsilon$ and ${\bf C}$    
    - Form **D** from occupied orbital slice of **C**

In [ ]:
# ==> SCF Iterations <==
# Pre-iteration energy declarations
SCF_E = 0.0
E_old = 0.0

print('==> Starting SCF Iterations <==\n')

# Begin Iterations
for scf_iter in range(1, MAXITER + 1):
    # Build Fock matrix

    
    # Compute RHF energy

    
    # SCF Converged?
    if (abs(SCF_E - E_old) < E_conv):
        break
    E_old = SCF_E
    
    # Compute new orbital guess

    
    # MAXITER exceeded?
    if (scf_iter == MAXITER):
        raise Exception("Maximum number of SCF iterations exceeded.")

# Post iterations
print('\nSCF converged.')
print('Final RHF Energy: %.8f [Eh]' % (SCF_E))

Congratulations! You've written your very own Restricted Hartree-Fock program!  Finally, let's check your final RHF energy against PySCF

In [ ]:
mf = scf.RHF(mol)
SCF_E_pyscf = mf.kernel()
np.isclose(SCF_E_pyscf, SCF_E, rtol=0., atol=1e-06)